#### 0. 라이브러리 import 및 설치

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install konlpy

In [ ]:
!pip install kiwipiepy

In [ ]:
!sudo apt-get update
!sudo apt-get install -y mecab libmecab-dev mecab-ipadic-utf8
!git clone --depth 1 https://github.com/gogumaUno/mecab-ko-dic.git
!bash mecab-ko-dic/tools/add-userdic.sh
!sudo cp -r mecab-ko-dic /usr/local/lib/mecab/dic/mecab-ko-dic
!pip install mecab-python3

In [ ]:
!pip install torchtext

In [ ]:
import pandas as pd
import numpy as np
import re

from konlpy.tag import Okt, Kkma, Hannanum, Komoran
from kiwipiepy import Kiwi

from sklearn.model_selection import train_test_split

# from torchtext.data import Field, TabularDataset, BucketIterator

import torch
from torch.utils.data import Dataset, DataLoader

# from torchtext.vocab import Vocab
from torchtext.vocab import build_vocab_from_iterator
from collections import Counter

#### A. 데이터 전처리

In [ ]:
standard = pd.read_parquet("/content/drive/MyDrive/2024-2 KUBIG/24S NLP Basic Team 4/data/경상도_학습데이터_대화_표준버전.parquet")
dialect = pd.read_parquet("/content/drive/MyDrive/2024-2 KUBIG/24S NLP Basic Team 4/data/경상도_학습데이터_대화_방언버전_방언개수추가.parquet")

In [ ]:
data = pd.DataFrame(columns=["speaker_1", "speaker_2", "speaker_2_eojeol_sum"])
data["speaker_1"] = list(standard["speaker_1"])
data["speaker_2"] = list(dialect["speaker_2"])
data["speaker_2_eojeol_sum"] = list(dialect["speaker_2_eojeol_sum"])

In [ ]:
data.head()

In [ ]:
# 구두점, 기호를 없애주고 소문자로 바꿔주는 함수
def remove_punc(string):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in string:
        if char not in punctuations:
            no_punct = no_punct + char  # space is also a character
    return no_punct

# 길이가 하나인 문자를 없애주는 함수
def del_one_char(string):
    result = []
    string_seq = string.split(" ")
    for s in string_seq:
        if len(s) != 1:
            result.append(s)
    return ' '.join(result)

def is_len_over_three(string):
    string_seq = string.split(" ")
    if len(string_seq) > 3:
        return True
    else:
        return False

In [ ]:
# 영어도 지울지는 잠시 보류
data["speaker_1"] = data["speaker_1"].apply(lambda x : remove_punc(x))
data["speaker_2"] = data["speaker_2"].apply(lambda x : remove_punc(x))

# 이 가설이 정확할지는 모르겠으나, "챗봇" 구현에 중요한 요소 중 하나는 얼마나 사람처럼 응답하는가 인 것 같아
# 한 글자짜리 단어를 모두 지울 시 어색해지지 않을까 하는 우려에서 우선 배제
# data["speaker_1"] = data["speaker_1"].apply(lambda x : del_one_char(x))
# data["speaker_2"] = data["speaker_2"].apply(lambda x : del_one_char(x))

# 질의의 단어 수가 충분치 않은 경우, 질문과 응답의 영양가가 없어보여 느낌상 3개 초과의 단어로 구성된 문장만 반영하도록
data = data[data["speaker_1"].apply(lambda x : is_len_over_three(x))]
data = data[data["speaker_2"].apply(lambda x : is_len_over_three(x))]

In [ ]:
# basic EDA

data_2 = data.copy()

data_2["speaker_1_length"] = data_2["speaker_1"].apply(lambda x : len(x.split(" ")))
data_2["speaker_2_length"] = data_2["speaker_2"].apply(lambda x : len(x.split(" ")))

speaker_1_length = list(data_2["speaker_1_length"])
speaker_2_length = list(data_2["speaker_2_length"])

quartiles_1 = np.percentile(speaker_1_length, [25, 50, 75])
quartiles_2 = np.percentile(speaker_2_length, [25, 50, 75])
quartiles_3 = np.percentile(data_2["speaker_2_eojeol_sum"], [25, 50, 75])

# 사분위수
print(quartiles_1)
print(quartiles_2)
print(quartiles_3)

print(len(data_2[data_2["speaker_2_eojeol_sum"] == 0]))

data_2 = data_2[data_2["speaker_2_eojeol_sum"] != 0]

#### B. Transformer

In [ ]:
# kiwi / okt / kkma / komoran / split (띄워쓰기)
# 총 5가지 종류의 tokenizer 를 만들어 테스트 진행

okt = Okt()
kkma = Kkma()
kiwi = Kiwi()
komoran = Komoran()

def tokenizer_kiwi(text):
  result = kiwi.analyze(text)
  kiwi_result = []

  for sentence in result:
    for word in sentence[0]:
      kiwi_result.append(word.form)

  return kiwi_result

def tokenizer_okt(text):
  return okt.morphs(text)

def tokenizer_kkma(text):
  return kkma.morphs(text)

def tokenizer_komoran(text):
  return komoran.morphs(text)

def tokenizer_split(text):
  return text.split(" ")


In [ ]:
# vocab (단어와 index 를 매핑해주는 사전) 생성 메서드
def build_vocab(dataframe, target_column, tokenizer, min_freq):
  tokenized_texts = []

  for idx, row in dataframe.iterrows():
    tokens = tokenizer(row[target_column])
    tokenized_texts.append(tokens)

  # counter = Counter(token for tokens in tokenized_texts for token in tokens)
  def yield_tokens():
    for tokens in tokenized_texts:
      yield tokens

  vocab = build_vocab_from_iterator(yield_tokens(), specials=['<unk>', '<pad>', '<sos>', '<eos>'], min_freq=min_freq)
  # unk 토큰 설정
  vocab.set_default_index(vocab['<unk>'])
  # vocab = Vocab(counter, specials=['<unk>', '<pad>', '<sos>', '<eos>'], min_freq=min_freq)
  return vocab

In [ ]:
# train / test data split

train, test = train_test_split(data_2, test_size=0.2, random_state=1097)

In [ ]:
# torch 의 dataset / dataloader
class CustomTextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, vocab_src, vocab_trg, max_len=30):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.vocab_src = vocab_src
        self.vocab_trg = vocab_trg
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src_text = self.data.iloc[idx, 0]
        trg_text = self.data.iloc[idx, 1]
        src_tokens = self.tokenizer(src_text)
        trg_tokens = self.tokenizer(trg_text)

        src_indexes = [self.vocab_src['<sos>']]
        trg_indexes = [self.vocab_trg['<sos>']]

        # 소스 토큰을 인덱스로 변환 (존재하지 않는 키는 <unk>으로 처리)
        for token in src_tokens[:self.max_len - 2]:
            try:
                src_indexes.append(self.vocab_src[token])
            except KeyError:
                src_indexes.append(self.vocab_src['<unk>'])

        # 타겟 토큰을 인덱스로 변환 (존재하지 않는 키는 <unk>으로 처리)
        for token in trg_tokens[:self.max_len - 2]:
            try:
                trg_indexes.append(self.vocab_trg[token])
            except KeyError:
                trg_indexes.append(self.vocab_trg['<unk>'])

        # 종료 토큰 추가
        src_indexes.append(self.vocab_src['<eos>'])
        trg_indexes.append(self.vocab_trg['<eos>'])

        if len(src_indexes) < self.max_len:
            src_indexes += [self.vocab_src['<pad>']] * (self.max_len - len(src_indexes))
        if len(trg_indexes) < self.max_len:
            trg_indexes += [self.vocab_trg['<pad>']] * (self.max_len - len(trg_indexes))

        return torch.tensor(src_indexes), torch.tensor(trg_indexes)
# collate_fn 이 max_len torch 길이를 맞춰주기 위한 역할인데,
# 여기서 두 가지 아이디어가 존재한다.
#   1. 단순 길이 기준으로 자르기
#   2. vocab 의 우선도 기준으로 자르기
# 만약 단순히 토큰의 최대 길이에 맞춰 padding 을 넣어주면 쓸데없이 크기만 커지는 걸 방지하기 위함

# 가변 배치 학습
# sampler 를 이용하면 되는데 이 부분은 우선 pass

In [ ]:
vocab_src = build_vocab(train, "speaker_1", tokenizer_okt, min_freq=2)
vocab_trg = build_vocab(train, "speaker_2", tokenizer_okt, min_freq=2)

In [ ]:
print(len(vocab_src))
print(len(vocab_trg))

In [ ]:
# Dataset 및 DataLoader
train_dataset = CustomTextDataset(train, tokenizer_okt, vocab_src, vocab_trg)
test_dataset = CustomTextDataset(test, tokenizer_okt, vocab_src, vocab_trg)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
# DataLoader 예시 확인
for i, (src_batch, trg_batch) in enumerate(train_loader):
    print(f"Batch {i+1}:")
    print(f"Source Batch:\n{src_batch}")
    print(f"Target Batch:\n{trg_batch}")
    print("\n")
    if i == 2:  # 3개의 배치만 출력
        break

In [ ]:
import torch.nn as nn

class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, dropout_ratio, device):
        super().__init__()

        assert hidden_dim % n_heads == 0

        self.hidden_dim = hidden_dim # 임베딩 차원
        self.n_heads = n_heads # 헤드(head)의 개수: 서로 다른 어텐션(attention) 컨셉의 수
        self.head_dim = hidden_dim // n_heads # 각 헤드(head)에서의 임베딩 차원

        self.fc_q = nn.Linear(hidden_dim, hidden_dim) # Query 값에 적용될 FC 레이어
        self.fc_k = nn.Linear(hidden_dim, hidden_dim) # Key 값에 적용될 FC 레이어
        self.fc_v = nn.Linear(hidden_dim, hidden_dim) # Value 값에 적용될 FC 레이어

        self.fc_o = nn.Linear(hidden_dim, hidden_dim)

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

    def forward(self, query, key, value, mask = None):

        batch_size = query.shape[0]

        # query: [batch_size, query_len, hidden_dim]
        # key: [batch_size, key_len, hidden_dim]
        # value: [batch_size, value_len, hidden_dim]

        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        # Q: [batch_size, query_len, hidden_dim]
        # K: [batch_size, key_len, hidden_dim]
        # V: [batch_size, value_len, hidden_dim]

        # hidden_dim → n_heads X head_dim 형태로 변형
        # n_heads(h)개의 서로 다른 어텐션(attention) 컨셉을 학습하도록 유도
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)

        # Q: [batch_size, n_heads, query_len, head_dim]
        # K: [batch_size, n_heads, key_len, head_dim]
        # V: [batch_size, n_heads, value_len, head_dim]

        # Attention Energy 계산
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

        # energy: [batch_size, n_heads, query_len, key_len]

        # 마스크(mask)를 사용하는 경우
        if mask is not None:
            # 마스크(mask) 값이 0인 부분을 -1e10으로 채우기
            energy = energy.masked_fill(mask==0, -1e10)

        # 어텐션(attention) 스코어 계산: 각 단어에 대한 확률 값
        attention = torch.softmax(energy, dim=-1)

        # attention: [batch_size, n_heads, query_len, key_len]

        # 여기에서 Scaled Dot-Product Attention을 계산
        x = torch.matmul(self.dropout(attention), V)

        # x: [batch_size, n_heads, query_len, head_dim]

        x = x.permute(0, 2, 1, 3).contiguous()

        # x: [batch_size, query_len, n_heads, head_dim]

        x = x.view(batch_size, -1, self.hidden_dim)

        # x: [batch_size, query_len, hidden_dim]

        x = self.fc_o(x)

        # x: [batch_size, query_len, hidden_dim]

        return x, attention

In [ ]:
# 포워드 레이어 (선형변환)

class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hidden_dim, pf_dim, dropout_ratio):
        super().__init__()

        self.fc_1 = nn.Linear(hidden_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hidden_dim)

        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, x):

        # x: [batch_size, seq_len, hidden_dim]

        x = self.dropout(torch.relu(self.fc_1(x)))

        # x: [batch_size, seq_len, pf_dim]

        x = self.fc_2(x)

        # x: [batch_size, seq_len, hidden_dim]

        return x

In [ ]:
# 인코더 레이어

class EncoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

    # 하나의 임베딩이 복제되어 Query, Key, Value로 입력되는 방식
    def forward(self, src, src_mask):

        # src: [batch_size, src_len, hidden_dim]
        # src_mask: [batch_size, src_len]

        # self attention
        # 필요한 경우 마스크(mask) 행렬을 이용하여 어텐션(attention)할 단어를 조절 가능
        _src, _ = self.self_attention(src, src, src, src_mask)

        # DH : forward 메서드를 호출할 필요가 없음

        # dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))

        # src: [batch_size, src_len, hidden_dim]

        # position-wise feedforward
        _src = self.positionwise_feedforward(src)

        # dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))

        # src: [batch_size, src_len, hidden_dim]

        return src

In [ ]:
# 인코더 모델 생성

class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=100):
        super().__init__()

        self.device = device

        self.tok_embedding = nn.Embedding(input_dim, hidden_dim)
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)

        # nn.Embedding : https://wikidocs.net/64779
        # 임베딩은 결국 룩업 테이블을 생성하는 것...!

        self.layers = nn.ModuleList([EncoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

        # nn.ModuleList : nn.Module 클래스를 확장하는 데에 사용하는 유틸리티
        # 여러 층으로 구성되어있는 Encoder 부분을 관리하기 쉽도록 활용함

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def forward(self, src, src_mask):

        # src: [batch_size, src_len]
        # src_mask: [batch_size, src_len]

        batch_size = src.shape[0]
        src_len = src.shape[1]

        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        # positional encoding

        # pos: [batch_size, src_len]

        # 소스 문장의 임베딩과 위치 임베딩을 더한 것을 사용
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))

        # note : 으음 이게 잘 되나?
        # 위치 정보가 충분히 반영되지 않을 것 같은 느낌.
        # nn.Emdedding 의 임베딩 테이블은 랜덤한 숫자들로 생성될텐데,,,

        # src: [batch_size, src_len, hidden_dim]

        # 모든 인코더 레이어를 차례대로 거치면서 순전파(forward) 수행
        for layer in self.layers:
            src = layer(src, src_mask)

        # src: [batch_size, src_len, hidden_dim]

        return src # 마지막 레이어의 출력을 반환

In [ ]:
# 디코더 레이어

class DecoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.encoder_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더의 출력 값(enc_src)을 어텐션(attention)하는 구조
    def forward(self, trg, enc_src, trg_mask, src_mask):

        # trg: [batch_size, trg_len, hidden_dim]
        # enc_src: [batch_size, src_len, hidden_dim]
        # trg_mask: [batch_size, trg_len]
        # src_mask: [batch_size, src_len]

        # self attention
        # 자기 자신에 대하여 어텐션(attention)
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)

        # dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))

        # trg: [batch_size, trg_len, hidden_dim]

        # encoder attention
        # 디코더의 쿼리(Query)를 이용해 인코더를 어텐션(attention)
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)

        # dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))

        # trg: [batch_size, trg_len, hidden_dim]

        # positionwise feedforward
        _trg = self.positionwise_feedforward(trg)

        # dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))

        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return trg, attention

In [ ]:
# 디코더 모델 생성

class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=100):
        super().__init__()

        self.device = device

        self.tok_embedding = nn.Embedding(output_dim, hidden_dim)
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)

        self.layers = nn.ModuleList([DecoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

        self.fc_out = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def forward(self, trg, enc_src, trg_mask, src_mask):

        # trg: [batch_size, trg_len]
        # enc_src: [batch_size, src_len, hidden_dim]
        # trg_mask: [batch_size, trg_len]
        # src_mask: [batch_size, src_len]

        batch_size = trg.shape[0]
        trg_len = trg.shape[1]

        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

        # pos: [batch_size, trg_len]

        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))

        # trg: [batch_size, trg_len, hidden_dim]

        for layer in self.layers:
            # 소스 마스크와 타겟 마스크 모두 사용
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)

        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        output = self.fc_out(trg)

        # output: [batch_size, trg_len, output_dim]

        return output, attention

In [ ]:
# 트랜스포머 모델

class Transformer(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    # 소스 문장의 <pad> 토큰에 대하여 마스크(mask) 값을 0으로 설정
    def make_src_mask(self, src):

        # src: [batch_size, src_len]

        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        # src_mask: [batch_size, 1, 1, src_len]

        return src_mask

    # 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용
    def make_trg_mask(self, trg):

        # trg: [batch_size, trg_len]

        """ (마스크 예시)
        1 0 0 0 0
        1 1 0 0 0
        1 1 1 0 0
        1 1 1 0 0
        1 1 1 0 0
        """
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)

        # trg_pad_mask: [batch_size, 1, 1, trg_len]

        trg_len = trg.shape[1]

        """ (마스크 예시)
        1 0 0 0 0
        1 1 0 0 0
        1 1 1 0 0
        1 1 1 1 0
        1 1 1 1 1
        """
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()

        # trg_sub_mask: [trg_len, trg_len]

        trg_mask = trg_pad_mask & trg_sub_mask

        # trg_mask: [batch_size, 1, trg_len, trg_len]

        return trg_mask

    def forward(self, src, trg):

        # src: [batch_size, src_len]
        # trg: [batch_size, trg_len]

        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)

        # src_mask: [batch_size, 1, 1, src_len]
        # trg_mask: [batch_size, 1, trg_len, trg_len]

        enc_src = self.encoder(src, src_mask)

        # enc_src: [batch_size, src_len, hidden_dim]

        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)

        # output: [batch_size, trg_len, output_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return output, attention

In [ ]:
# 모델 튜닝

INPUT_DIM = len(vocab_src)
OUTPUT_DIM = len(vocab_trg)
HIDDEN_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

In [ ]:
# SRC_PAD_IDX = vocab_src.get_stoi[vocab_src.pad_token]
# TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

SRC_PAD_IDX = vocab_src["<pad>"]
TRG_PAD_IDX = vocab_trg["<pad>"]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, HIDDEN_DIM, ENC_LAYERS, ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, HIDDEN_DIM, DEC_LAYERS, DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, device)

# Transformer 객체 선언
model = Transformer(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [ ]:
# 파라미터 개수

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

In [ ]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
# 모델 학습(train) 함수

def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0

    # 전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch[0].to(device)
        trg = batch[1].to(device)

        optimizer.zero_grad()

        # 출력 단어의 마지막 인덱스(<eos>)는 제외
        # 입력을 할 때는 <sos>부터 시작하도록 처리
        output, _ = model(src, trg[:,:-1])

        # output: [배치 크기, trg_len - 1, output_dim]
        # trg: [배치 크기, trg_len]

        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        # 출력 단어의 인덱스 0(<sos>)은 제외
        trg = trg[:,1:].contiguous().view(-1)

        # output: [배치 크기 * trg_len - 1, output_dim]
        # trg: [배치 크기 * trg len - 1]

        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산

        # 기울기(gradient) clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # 파라미터 업데이트
        optimizer.step()

        # 전체 손실 값 계산
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:

# 모델 평가(evaluate) 함수

def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0

    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch[0]
            trg = batch[1]

            # 출력 단어의 마지F막 인덱스(<eos>)는 제외
            # 입력을 할 때는 <sos>부터 시작하도록 처리
            output, _ = model(src, trg[:,:-1])

            # output: [배치 크기, trg_len - 1, output_dim]
            # trg: [배치 크기, trg_len]

            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            # 출력 단어의 인덱스 0(<sos>)은 제외
            trg = trg[:,1:].contiguous().view(-1)

            # output: [배치 크기 * trg_len - 1, output_dim]
            # trg: [배치 크기 * trg len - 1]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
import time
import math
import random

N_EPOCHS = 10
CLIP = 1

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_loader, optimizer, criterion, CLIP)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')

In [ ]:
torch.save(model.state_dict(), 'transformer_model_tokenizer_okt_dialect_1.pth')

#### C. T5 model + finetuning

In [ ]:
import pandas as pd
import torch

from transformers import T5TokenizerFast, T5ForConditionalGeneration
from transformers import AdamW

from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset

# 디바이스 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")


# 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        questions = self.data.iloc[idx]['speaker_1']
        answers = self.data.iloc[idx]['speaker_2']

        # 토큰 수를 max_length에 맞춰서 자르거나 패딩
        inputs = self.tokenizer(questions, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        labels = self.tokenizer(answers, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')

        return {
            'input_ids': inputs.input_ids[0],
            'labels': labels.input_ids[0]
        }

# 토크나이저와 모델 로드
# tokenizer = T5TokenizerFast.from_pretrained('paust/pko-chat-t5-base')
# model = T5ForConditionalGeneration.from_pretrained('paust/pko-chat-t5-base')

tokenizer = T5TokenizerFast.from_pretrained('paust/pko-t5-base')
model = T5ForConditionalGeneration.from_pretrained('paust/pko-t5-base')
model.to(device)


# tokenizer = T5TokenizerFast.from_pretrained('paust/pko-t5-base')
# model = T5ForConditionalGeneration.from_pretrained('paust/pko-t5-base')

# 학습 설정
optimizer = AdamW(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

num_epochs = 5
log_interval = 100
checkpoint_interval = 0.5  # 0.5 에폭마다 체크포인트 저장

# DataLoader를 사용하여 데이터 로드
batch_size = 4
dataset = CustomDataset(data_2, tokenizer)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 학습 루프에서 데이터 로더 사용
total_steps = 0
for epoch in range(num_epochs):
    for batch in tqdm(data_loader, total=len(data_loader)):
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        # 모델 학습
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss

        # 역전파 및 가중치 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_steps += 1

        # 0.5 에폭마다 체크포인트 저장
        if total_steps % int(len(dataset) / (batch_size * 2)) == 0:
            checkpoint_dir = f"checkpoint_{total_steps}"
            model.save_pretrained(checkpoint_dir)

In [ ]:
# 모델 상태 사전을 저장할 경로 지정
save_path = '/kaggle/working/t5_finetuning_3.pth'

# 모델 상태 사전 저장
torch.save(model.state_dict(), save_path)

#### D. KoGPT2 + finetuning

In [ ]:
import math
import numpy as np
import pandas as pd

import random
import re
import torch
import urllib.request

from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

In [ ]:
BOS = "</s>"
EOS = "</s>"
PAD = "<pad>"
MASK = "<unused0>"
UNK = "<unk>"

Q_TKN = "<usr>"
A_TKN = "<sys>"
SENT = "<unused1>"

# 허깅페이스 transformers 에 등록된 사전 학습된 koGTP2 토크나이저를 가져온다.
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token=BOS, eos_token=EOS, unk_token=UNK, pad_token=PAD, mask_token=MASK,)
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [ ]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=128):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]

        q = turn["speaker_1"]  # 질문을 가져온다.
        a = turn["speaker_2"]  # 답변을 가져온다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        # #질문+답변, 마스크, 답변
        # return (token_ids, np.array(mask), labels_ids)

        return torch.LongTensor(token_ids), torch.LongTensor(mask), torch.LongTensor(labels_ids)

In [ ]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]

    # 각 리스트를 텐서로 변환하고 배치 차원 추가
    data = torch.stack(data)
    mask = torch.stack(mask)
    label = torch.stack(label)

    return data, mask, label

In [ ]:
train_set = ChatbotDataset(data_2, max_len=128)

#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=8, num_workers=2, shuffle=True, collate_fn=collate_batch,)

In [ ]:
print("start")
for batch_idx, samples in enumerate(train_dataloader):
    token_ids, mask, label = samples
    print("token_ids ====> ", token_ids)
    print("mask =====> ", mask)
    print("label =====> ", label)
    break
print("end")

In [ ]:
model.to(device)
model.train()

In [ ]:
# 하이퍼 파라미터 설정

learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

num_epoch = 5
Sneg = -1e18

In [ ]:
from tqdm import tqdm  # tqdm 모듈 임포트

total_steps = 0
for epoch in range(num_epoch):
    print(f"Epoch {epoch+1}/{num_epoch}")

    # tqdm을 사용하여 진행 상황 표시
    for batch_idx, samples in enumerate(tqdm(train_dataloader, total=len(train_dataloader))):
        optimizer.zero_grad()

        # GPU로 텐서를 이동
        token_ids, mask, label = [tensor.to(device) for tensor in samples]

        # 모델의 출력 계산
        out = model(token_ids)
        out = out.logits  # logits 계산

        # 마스크 적용
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))

        # 손실 계산
        loss = criterion(mask_out.transpose(2, 1), label)
        avg_loss = loss.sum() / mask.sum()

        # 역전파 및 가중치 업데이트
        avg_loss.backward()
        optimizer.step()

        total_steps += 1

        # 0.5 에폭마다 체크포인트 저장
        if total_steps % int(len(train_dataloader) / 2) == 0:
            checkpoint_dir = f"checkpoint_{total_steps}"
            model.save_pretrained(checkpoint_dir)
            print(f"Checkpoint saved at step {total_steps}")

print("Training complete.")

In [ ]:
# 모델 상태 사전을 저장할 경로 지정
save_path = '/kaggle/working/kogpt2_finetuning.pth'

# 모델 상태 사전 저장
torch.save(model.state_dict(), save_path)